<a href="https://colab.research.google.com/github/DeisyData/BIT_IA_Bootcamp/blob/main/S11_C1_Emojis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Notebook to release Sentiment Analysis - prediction
# Iván Andrés Trujillo abella
# ivantrujillo1229@gmail.com

In [ ]:
url = "https://raw.githubusercontent.com/it-ces/Datasets/refs/heads/main/tweets(ai-page).csv"

In [ ]:
pip install emoji --upgrade

In [ ]:
import emoji
import pandas as pd
import numpy as np

In [ ]:
s = '😡'
code = 'U000{:X}'.format(ord(s))
print(code)

In [ ]:
matches = ["U0001F621", "U0001F607"]
emojis = [emoji.replace('U', r"\U").encode().decode('unicode-escape') for emoji in matches]
print(emojis)

In [ ]:
print(emoji.is_emoji("\U0001F607"))
emoji.is_emoji("😡")

In [ ]:
emoji.distinct_emoji_list("\U0001F607 hello world")

In [ ]:
df = pd.read_csv(url)

In [ ]:
df

In [ ]:
df['emoticons']='None'
for index in df.index:
    if len(emoji.distinct_emoji_list(df.loc[index,'tweet']))>0:
        im = emoji.distinct_emoji_list(df.loc[index,'tweet'])
        df.loc[index, 'emoticons'] = im[0] # take the first emoticon

df  = df[df['emoticons']!='None'].reset_index(drop=True)

def label_emoji(emojis):
    labels={'Angry':['🤬','😡', '😤','🤭'],
            'Smile':['😁','🤗', '😀'],
            'Nice':['😎','♥️','👍', '😍','👍🏾','😻','🙏🏽','🙌🏼',
                     '👏🏻','🦾', '🤘', '💪'],
            'No nice': ['🖕🏼','🤡','🤦','👎🏾','🥲' ,'😰'],
            'Sad':['😔', '😩','😢','😫', '😭' ]}
    class_name = 'Another'
    for emoticon in emojis:
        if emojis =='None':
            return 'None'
        for label in labels:
            if emoticon in labels[label]:
                class_name = label
    return class_name

In [ ]:
df['y'] = df['emoticons'].apply(lambda x: label_emoji(x))

In [ ]:
## Using regular expressions to tackle
df['tweet'] = df['tweet'].apply(lambda x : emoji.replace_emoji(x, replace=''))
df['tweet'] = df['tweet'].apply(lambda x : x.lower())
df['tweet'] =df['tweet'].replace('mi[Bb]anco', "", regex=True)
regex_url = "https?:\\/\\/(?:www\\.)?[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)$"
df['tweet'] = df['tweet'].replace(regex_url, "", regex=True)
df['tweet'] = df['tweet'].replace(r'\bhttps://t.co/[a-z0-9]*\b', "", regex=True)
df['tweet'] = df['tweet'].replace(r'[\d@#]', "", regex=True)

In [ ]:
df

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

## To modeling
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from wordcloud import WordCloud


# Logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score

import pandas as pd
import matplotlib.pyplot as plt
from sklearn import model_selection
import re

# Random Forest
from sklearn.ensemble import RandomForestClassifier



In [ ]:
df = df[df['y']!='Another']
df.reset_index(inplace=True, drop=True )

In [ ]:
df['y'].value_counts()

In [ ]:
df['feeling'] = np.where((df['y']=='Smile') | (df['y']=='Nice'), 'Positive', 'Negative')

In [ ]:
df['feeling'].value_counts()

In [ ]:
df

In [ ]:
emojis = df['emoticons'].unique()
base_of_text={}
for emo in emojis:
    base_of_text[emo] = "".join(df[df['emoticons']==emo]['tweet'].values)

In [ ]:
df['emoticons'].unique()

In [ ]:
wc = WordCloud(background_color="white", repeat=True)
wc.generate(base_of_text['😡'])
plt.axis("off")
plt.imshow(wc, interpolation="bilinear")
plt.show()

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords.words('spanish')

In [ ]:
en_stops = set(stopwords.words('spanish'))
en_stops

In [ ]:
all_words = ['There', 'is', 'a', 'tree','near','the','river', 'cómo','tan', 'alguna', 'casi', 'vaya']
for word in all_words:
    if word not in en_stops:
        print(word)
custom_stop_words = ['alguna', 'near', 'cómo',  'casi', 'vaya', ',']  # Replace with your custom stop words
en_stops.update(custom_stop_words)
for word in all_words:
    if word not in en_stops:
        print(word, 'added')

In [ ]:
df

In [ ]:
tweet  =  df.loc[0,'tweet']

In [ ]:
custom_stop_words = ['casi', 'pronto', 'tan',  'seguro', 'cúando']  # add to improve!
en_stops.update(custom_stop_words)

def remove_stopwords(tweet):
    new = []
    for word in tweet.split():
        if word not in en_stops:
            new.append(word)
    return " ".join(new)

In [ ]:
remove_stopwords('cada día expreso mi inconformidad con la entidad')

In [ ]:
df['tweet'] =  df['tweet'].apply(lambda x : remove_stopwords(x))

In [ ]:
emojis = df['emoticons'].unique()
base_of_text={}
for emo in emojis:
    base_of_text[emo] = "".join(df[df['emoticons']==emo]['tweet'].values)

In [ ]:
wc = WordCloud(background_color="white", repeat=True)
wc.generate(base_of_text['😡'])
plt.axis("off")
plt.imshow(wc, interpolation="bilinear")
plt.show()

In [ ]:
def remove_laughs(tweet):
    new = []
    for word in tweet.split():
        if bool(re.fullmatch('a*ja+j[ja]*', word))==False:
            new.append(word)
    return ' '.join(new)

In [ ]:
df['tweet'] = df['tweet'].apply(lambda x : remove_laughs(x))

In [ ]:
def remove_term(regex, tweet):
    new = []
    for word in tweet.split():
        if bool(re.fullmatch(regex, word))==False:
            new.append(word)
    return ' '.join(new)

In [ ]:
remove_term('app', 'app no funciona')

In [ ]:
df['tweet'] =  df['tweet'].apply(lambda x: remove_term('miB[b]anco',x))

In [ ]:
emojis = df['emoticons'].unique()
base_of_text={}
for emo in emojis:
    base_of_text[emo] = "".join(df[df['emoticons']==emo]['tweet'].values)
wc = WordCloud(background_color="white", repeat=True)
wc.generate(base_of_text['😡'])
plt.axis("off")
plt.imshow(wc, interpolation="bilinear")
plt.show()

In [ ]:
df['feeling'] = np.where(df['feeling']=='Positive',1 ,0)
X  = df['tweet']
y = df['feeling']
X_train, X_test, y_train, y_test  = model_selection.train_test_split(X,y, test_size = 0.25,  random_state=666)

In [ ]:
df= df[df['tweet'].apply(lambda tweet: len(str(tweet))!=0)]
df.reset_index(inplace=True, drop=True)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
### Logistic Regression
def grid_search_lr(X_train, y_train):
    model = LogisticRegression(random_state=666, max_iter=1000)
    class_weight =  [{1:0.5, 0:0.5}]
    solvers = ['liblinear']
    penalty = ['l2','l1', ]
    c_values = [1000, 100,50, 10,1,0.1 ]
    grid = dict(solver=solvers,penalty=penalty,C=c_values, class_weight= class_weight)
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
    scorer = make_scorer(f1_score, average = 'weighted')
    grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv,
                           scoring=scorer,error_score=0)
    grid_result = grid_search.fit(X_train, y_train)
    best_params = grid_result.best_params_
    return best_params

In [ ]:
# vectorization
vect = CountVectorizer().fit(X_train)
vect.get_feature_names_out()
X_train_vectorized = vect.transform(X_train)
params = grid_search_lr(X_train_vectorized, y_train)

# We are going to predict
model = LogisticRegression(penalty = params['penalty'],
                           class_weight=params['class_weight'],
                           solver = params['solver'],
                           C = params['C'],
                           random_state =123)
print(params)

In [ ]:
model.fit(X_train_vectorized, y_train)
preds = model.predict(vect.transform(X_test))
print(classification_report(y_test, preds))
import numpy as np
f_names = np.array(vect.get_feature_names_out())
sorted_coef  = model.coef_[0].argsort()
print(f_names[sorted_coef][0:5])

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
X_train = pd.DataFrame(X_train)
rus = RandomUnderSampler(random_state=1234)
X_train, y_train = rus.fit_resample(X_train, y_train)
X_train = X_train['tweet']

In [ ]:
vect = CountVectorizer().fit(X_train)
#vect.get_feature_names_out()
X_train_vectorized = vect.transform(X_train)
model.fit(X_train_vectorized, y_train)
preds = model.predict(vect.transform(X_test))
print(classification_report(y_test, preds))
import numpy as np
f_names = np.array(vect.get_feature_names_out())
sorted_coef  = model.coef_[0].argsort()
print(f_names[sorted_coef][0:5])

In [ ]:
def grid_RandomForest(X_train, y_train):
  model = RandomForestClassifier(random_state=0)
  n_estimators = np.arange(10,100,1)
  criterion = ['gini', 'entropy', 'log_loss']
  min_samples_split = [0.05, 0.1,]
  max_depth = [2,3,4,10]
  grid = dict(n_estimators = n_estimators, criterion = criterion,
              min_samples_split = min_samples_split, max_depth = max_depth)
  cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
  grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv,
                            scoring='f1',error_score='raise')
  grid_result = grid_search.fit(X_train, y_train)
  return  grid_result.best_estimator_

In [ ]:
best_model = grid_RandomForest(X_train_vectorized, y_train)
preds = best_model.predict(vect.transform(X_test))
print(classification_report(y_test, preds))